In [1]:
googlecolab = True

if googlecolab:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
    !pip install gym
    !pip install ludwig

  ERROR: HTTP error 403 while getting http://download.pytorch.org/whl/cu101/torch-0.4.1-cp36-cp36m-linux_x86_64.whl
  ERROR: Could not install requirement torch==0.4.1 from http://download.pytorch.org/whl/cu101/torch-0.4.1-cp36-cp36m-linux_x86_64.whl because of error 403 Client Error: Forbidden for url: http://download.pytorch.org/whl/cu101/torch-0.4.1-cp36-cp36m-linux_x86_64.whl
ERROR: Could not install requirement torch==0.4.1 from http://download.pytorch.org/whl/cu101/torch-0.4.1-cp36-cp36m-linux_x86_64.whl because of HTTP error 403 Client Error: Forbidden for url: http://download.pytorch.org/whl/cu101/torch-0.4.1-cp36-cp36m-linux_x86_64.whl for URL http://download.pytorch.org/whl/cu101/torch-0.4.1-cp36-cp36m-linux_x86_64.whl
     |████████████████████████████████| 174kB 3.4MB/s 
     |████████████████████████████████| 109.2MB 31kB/s 
     |████████████████████████████████| 491kB 10.9MB/s 
     |████████████████████████████████| 3.2MB 10.7MB/s 
  Created wheel for ludwig: filename=l

#### Importation du gridworld à partir du local

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls drive/'My Drive'/gridworld/

gridworld_env.py  gridworldPlans  __init__.py  __pycache__


In [0]:
cp -r drive/'My Drive'/gridworld/ ./

## Import packages

In [0]:
import matplotlib

#matplotlib.use("TkAgg")
import gym
!pip install gridworld
import gridworld
from gym import wrappers, logger
import numpy as np
import copy
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
import time

     |████████████████████████████████| 8.7MB 14.7MB/s 
     |████████████████████████████████| 62.2MB 46kB/s 
  Created wheel for gridworld: filename=gridworld-0.1-cp36-none-any.whl size=4347 sha256=b1a3eb91924ee15a97d3f44fc247357ab1235abcf9508f807fbd63fbcfc8e2be
  Stored in directory: /root/.cache/pip/wheels/61/50/33/8859b5be280b8b2b2f0dae3ff8495a025418c8adac718f46d4
Successfully built gridworld


/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

In [0]:
cuda=True

device = torch.device("cuda:0" if cuda else "cpu")
print(device)

cuda:0


## Classe NN


In [0]:
class NN(nn.Module):
    def __init__(self, inSize, outSize, layers=[]):
        super(NN,self).__init__()
        self.layers = nn.ModuleList([])
        for x in layers:
            self.layers.append(nn.Linear(inSize,x))
            inSize = x
        self.layers.append(nn.Linear(inSize,outSize))

    def forward(self, x):
        x = self.layers[0](x)
        for i in  range(1,len(self.layers)):
            x = torch.nn.functional.prelu(x, torch.tensor(0.2).to(device))
            x = self.layers[i](x)
        return x  

# Models

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torchsummary import summary

torch.manual_seed(999)

def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)

class ActorNetwork(nn.Module):
    """
    Actor (Policy) Network.
    """

    def __init__(self, state_dim, action_dim):
        """Initialize parameters and build model.
        :state_dim (int): Dimension of each state
        :action_dim (int): Dimension of each action
        """

        super(ActorNetwork, self).__init__()
    
        self.fc1 = nn.Linear(state_dim, 64)
        
        self.fc2 = nn.Linear(64, 128)
        
        self.fc3 = nn.Linear(128, action_dim)
        self.reset_parameters()

    def reset_parameters(self):
        """
        Initialize parameters
        """
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)

    def forward(self, x):
        """
        Maps a state to actions
        """
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return F.tanh(self.fc3(x))


class CriticNetwork(nn.Module):
    """
    Critic (State-Value) Network.
    """

    def __init__(self, state_dim, action_dim):
        """
        Initialize parameters and build model
        :state_dim (int): Dimension of each state
        :action_dim (int): Dimension of each action
        """
        super(CriticNetwork, self).__init__()
        self.state_fc = nn.Linear(state_dim, 64)
        self.fc1 = nn.Linear(action_dim+64, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
        self.reset_parameters()

    def reset_parameters(self):
        """
        Initialize parameters
        """
        self.state_fc.weight.data.uniform_(*hidden_init(self.state_fc))
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)

    def forward(self, state, action):
        """
        Maps a state-action pair to Q-values
        """
        state, action = state.squeeze(), action.squeeze()
        x = F.relu(self.state_fc(state))
        x = torch.cat((x, action), dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)
    
if __name__ == "__main__":
    # summarize network structures using torchsummary
    state_dim, action_dim = 24, 2
    actor = ActorNetwork(state_dim, action_dim).to(device)
    critic = CriticNetwork(state_dim, action_dim).to(device)
    sum_res = summary(actor, (1, state_dim))
    sum_res = summary(critic, [(1, state_dim), (1, action_dim)])

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 64]           1,600
            Linear-2               [-1, 1, 128]           8,320
            Linear-3                 [-1, 1, 2]             258
Total params: 10,178
Trainable params: 10,178
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.04
Estimated Total Size (MB): 0.04
----------------------------------------------------------------
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]           1,600
            Linear-2                  [-1, 128]           8,576
            Linear-3                   [-1, 64]           8,256
            Linear-4                    [-1

# MADDP Model

In [0]:
import torch
import torch.optim as optim
import torch.nn.functional as F
 
from collections import deque, namedtuple
import random
import copy

eps = np.finfo(np.float32).eps.item()


class MADDPG:
    """
    The Multi-Agent consisting of two DDPG Agents
    """
    def __init__(self,state_dim,
                 action_dim,
                 lr_actor,
                 lr_critic,
                 lr_decay,
                 replay_buff_size,
                 gamma,
                 batch_size,
                 random_seed, 
                 soft_update_tau):
    
        
        super(MADDPG, self).__init__()
        self.adversarial_agents = []
        for i in range(len(state_dim)):
          agent = DDPGAgent(state_dim[i],
                 action_dim,
                 )
          self.adversarial_agents.append(agent)
     # the agent self-plays with itself
        
    def get_actors(self):
        """
        get actors of all the agents in the MADDPG object
        """
        actors = [ddpg_agent.actor_local for ddpg_agent in self.adversarial_agents]
        return actors

    def get_target_actors(self):
        """
        get target_actors of all the agents in the MADDPG object
        """
        target_actors = [ddpg_agent.actor_target for ddpg_agent in self.adversarial_agents]
        return target_actors

    def act(self, states_all_agents, add_noise=False):
        """
        get actions from all agents in the MADDPG object
        """
        actions = [agent.act(state, add_noise) for agent, state in zip(self.adversarial_agents, states_all_agents)]
        return np.stack(actions, axis=0)

    def update(self, *experiences):
        """
        update the critics and actors of all the agents
        """
        states, actions, rewards, next_states, dones = experiences
        for agent_idx, agent in enumerate(self.adversarial_agents):
            state = states[agent_idx]
            action = actions[agent_idx]
            reward = rewards[agent_idx]
            next_state = next_states[agent_idx]
            done = dones[agent_idx]
            agent.update_model(state, action, reward, next_state, done)
            
    def save(self, path):
        """
        Save the model
        """
        agent = self.adversarial_agents[0]
        torch.save((agent.actor_local.state_dict(), agent.critic_local.state_dict()), path)
        
    def load(self, path):
        """
        Load model and decay learning rate
        """
        actor_state_dict, critic_state_dict = torch.load(path)
        agent = self.adversarial_agents[0]
        agent.actor_local.load_state_dict(actor_state_dict)
        agent.actor_target.load_state_dict(actor_state_dict)
        agent.critic_local.load_state_dict(critic_state_dict)
        agent.critic_target.load_state_dict(critic_state_dict)
        agent.lr_actor *= agent.lr_decay
        agent.lr_critic *= agent.lr_decay
        for group in agent.actor_optimizer.param_groups:
            group['lr'] = agent.lr_actor
        for group in agent.critic_optimizer.param_groups:
            group['lr'] = agent.lr_critic
        
        for i in range(1, len(self.adversarial_agents)):
            self.adversarial_agents[i] = agent
            
        print("Loaded models!")
            

class DDPGAgent:
    """
    A DDPG Agent
    """    
    def __init__(self,
                 state_dim,
                 action_dim,
                 lr_actor = 1e-4,
                 lr_critic = 1e-4,
                 lr_decay = .95,
                 replay_buff_size = 10000,
                 gamma = .9,
                 batch_size = 128,
                 random_seed = 42,
                 soft_update_tau = 1e-3
                 ):
        """
        Initialize model
        """
        self.lr_actor = lr_actor
        self.gamma = gamma
        self.lr_critic = lr_critic
        self.lr_decay = lr_decay
        self.tau = soft_update_tau
        self.actor_local = ActorNetwork(state_dim, action_dim).to(device=device)
        self.actor_target = ActorNetwork(state_dim, action_dim).to(device=device)
        self.actor_optimizer = optim.Adam(self.actor_local.parameters(), lr=self.lr_actor)
        
        self.critic_local = CriticNetwork(state_dim, action_dim).to(device=device)
        self.critic_target = CriticNetwork(state_dim, action_dim).to(device=device)
        self.critic_optimizer = optim.Adam(self.critic_local.parameters(), lr=self.lr_critic)
        
        self.noise = OUNoise(action_dim, random_seed)

        self.memory = ReplayBuffer(action_dim, replay_buff_size, batch_size, random_seed)
  
        
    def update_model(self, state, action, reward, next_state, done):
        """
        Update policy and value parameters using given batch of experience tuples.
        Q_targets = r + γ * critic_target(next_state, actor_target(next_state))
        where:
            actor_target(state) -> action
            critic_target(state, action) -> Q-value
        
        :experiences (Tuple[torch.Tensor]): tuple of (s, a, r, s', done) tuples 
        :gamma (float): discount factor
        """
        self.memory.add(state, action, reward, next_state, done)
        if not self.memory.is_ready():
            return
        
        experiences = self.memory.sample()
        states, actions, rewards, next_states, dones = experiences

        # ---------------------------- update critic ---------------------------- #
        # Get predicted next-state actions and Q values from target models
        actions_next = self.actor_target(next_states)
        Q_targets_next = self.critic_target(next_states, actions_next)
        # Compute Q targets for current states (y_i)
        Q_targets = rewards + (self.gamma * Q_targets_next * (1 - dones)).detach()
        # Compute critic loss
        Q_expected = self.critic_local(states, actions)
        y = Q_targets.mean().item()

        critic_loss = F.smooth_l1_loss(Q_expected, Q_targets)
        # Minimize the loss
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        #--------------------------- PLOTING -----------------------------------#
    
        
        # ---------------------------- update actor ---------------------------- #
        # Compute actor loss
        actions_pred = self.actor_local(states)
        actor_loss = -self.critic_local(states, actions_pred).mean()
        # Minimize the loss
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        # ----------------------- update target networks ----------------------- #
        self.soft_update(self.critic_local, self.critic_target, self.tau)
        self.soft_update(self.actor_local, self.actor_target, self.tau)   
        
    def act(self, state, noise_t=0.0):
        """
        Returns actions for given state as per current policy.
        """
        if random.random()<0.1:
          v=random.random()
          return np.array([v,1-v])
        if len(np.shape(state)) == 1:
            state = state.reshape(1,-1)
        state = torch.from_numpy(state).float().to(device=device)
        self.actor_local.eval()
        with torch.no_grad():
            action = self.actor_local(state).cpu().data.numpy()
        self.actor_local.train()
        action += self.noise.sample() * noise_t
        return np.clip(action, -1, 1).squeeze()
    
    def reset(self):
        self.noise.reset()
        
    def soft_update(self, local_model, target_model, tau):
        """
        Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target
        :local_model: PyTorch model (weights will be copied from)
        :target_model: PyTorch model (weights will be copied to)
        :tau (float): interpolation parameter 
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)
            

class OUNoise:
    """
    Ornstein-Uhlenbeck process.
    """

    def __init__(self, size, seed, mu=0., theta=0.15, sigma=0.2):
        """
        Initialize parameters and noise process."""
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.seed = random.seed(seed)
        self.reset()

    def reset(self):
        """
        Reset the internal state (= noise) to mean (mu).
        """
        self.state = copy.copy(self.mu)

    def sample(self):
        """
        Update internal state and return it as a noise sample.
        """
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.array([random.random() for i in range(len(x))])
        self.state = x + dx
        return self.state


class ReplayBuffer:
    """
    Fixed-size buffer to store experience tuples.
    """

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """
        Initialize a ReplayBuffer object.
        
        :buffer_size (int): maximum size of buffer
        :batch_size (int): size of each training batch
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  # internal memory (deque)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, done):
        """
        Add a new experience to memory.
        """
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """
        Randomly sample a batch of experiences from memory.
        """
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)

        return (states, actions, rewards, next_states, dones)
    
    def is_ready(self):
        return len(self.memory) > self.batch_size

    def __len__(self):
        """
        Return the current size of internal memory.
        """
        return len(self.memory)

# Main

In [0]:
!pip install matplotlib
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

def draw(scores, path="fig.png", title="Performance", xlabel="Episode #", ylabel="Score"):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.title(title)
    plt.plot(np.arange(len(scores)), scores)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    plt.savefig(path)

In [0]:
!wget "http://dac.lip6.fr/master/wp-content/uploads/2019/12/multiagent.zip"
!unzip /content/multiagent.zip
!rm /content/multiagent.zip

--2020-02-15 21:07:05--  http://dac.lip6.fr/master/wp-content/uploads/2019/12/multiagent.zip
Resolving dac.lip6.fr (dac.lip6.fr)... 132.227.201.33
Connecting to dac.lip6.fr (dac.lip6.fr)|132.227.201.33|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51346 (50K) [application/zip]
Saving to: ‘multiagent.zip’

multiagent.zip      100%[===================>]  50.14K   186KB/s    in 0.3s    

2020-02-15 21:07:06 (186 KB/s) - ‘multiagent.zip’ saved [51346/51346]

Archive:  /content/multiagent.zip
   creating: multiagent/
  inflating: multiagent/multi_discrete.py  
  inflating: multiagent/rendering.py  
  inflating: multiagent/environment.py  
  inflating: multiagent/reraise.py   
  inflating: multiagent/scenario.py  
  inflating: multiagent/__init__.py  
  inflating: multiagent/policy.py    
  inflating: multiagent/core.py      
   creating: multiagent/__pycache__/
  inflating: multiagent/__pycache__/rendering.cpython-37.pyc  
  inflating: multiagent/__pycache__/multi

In [0]:
!ls /content/

drive  gridworld  multiagent  sample_data


In [0]:
import numpy as np
#from agent import MADDPG
#from utils import draw

import matplotlib

import gym
import multiagent
import multiagent.scenarios
import multiagent.scenarios.simple_tag as simple_tag
import multiagent.scenarios.simple_tag as simple_spread
import multiagent.scenarios.simple_tag as simple_adversary
from multiagent.environment import MultiAgentEnv
import multiagent.scenarios as scenarios
from gym import wrappers, logger
import numpy as np
import copy


unity_environment_path = "./Tennis_Linux/Tennis.x86_64"
best_model_path = "./best_model.checkpoint" 
rollout_length = 3

def make_env(scenario_name, benchmark=False):
    '''
    Creates a MultiAgentEnv object as env. This can be used similar to a gym
    environment by calling env.reset() and env.step().
    Use env.render() to view the environment on the screen.

    Input:
        scenario_name   :   name of the scenario from ./scenarios/ to be Returns
                            (without the .py extension)
        benchmark       :   whether you want to produce benchmarking data
                            (usually only done during evaluation)

    Some useful env properties (see environment.py):
        .observation_space  :   Returns the observation space for each agent
        .action_space       :   Returns the action space for each agent
        .n                  :   Returns the number of Agents
    '''
    from multiagent.environment import MultiAgentEnv
    import multiagent.scenarios as scenarios

    # load scenario from script
    scenario = scenarios.load(scenario_name + ".py").Scenario()
    # create world
    world = scenario.make_world()
    # create multiagent environment
    world.dim_c = 0
    if benchmark:
        env = MultiAgentEnv(world, scenario.reset_world, scenario.reward, scenario.observation, scenario.benchmark_data)
    else:
        env = MultiAgentEnv(world, scenario.reset_world, scenario.reward, scenario.observation)
    env.discrete_action_space = False
    env.discrete_action_input = False
    scenario.reset_world(world)
    return env,scenario,world

if __name__ == "__main__":
    # prepare environment
    env,scenario,world = make_env('simple_spread')

    
    num_agents = len(env.agents)
    print('Number of agents:', num_agents)
    


    num_episodes = 300
    
    state_size = env.observation_space
    action_size = 2

    o=env.reset()
    state_size=[len(i) for i in o]

    agent = MADDPG(state_size, 
                   action_size,
                   lr_actor = 1e-5,
                   lr_critic = 1e-4,
                   lr_decay = .995,
                   replay_buff_size = int(1e6),
                   gamma = .95,
                   batch_size = 64,
                   random_seed = 999,
                   soft_update_tau = 1e-3
                 )
    total_rewards = []
    avg_scores = []
    max_avg_score = -1
    max_score = -1
    threshold_init = 20
    noise_t = 1.0
    noise_decay = .995
    worsen_tolerance = threshold_init  # for early-stopping training if consistently worsen for # episodes
    reward = 0
    num_agents = 3
    for i_episode in range(1, num_episodes+1):
        obs = env.reset()   # reset the environment        
        scores = np.zeros(num_agents)                        # initialize score array
        dones = [False]*num_agents
        #while not np.any(dones):
        for i in range(100):
          if np.any(dones):
            break
          actions = agent.act(obs,noise_t)              # select an action
          next_states , rewards, dones, _ = env.step(actions)         # send the action to the environment     
          
          agent.update(obs, actions, rewards, next_states, dones)
          noise_t *= noise_decay
          scores += rewards       
          obs = next_states                         
          # update scores 
        
        episode_score = np.max(scores)
        total_rewards.append(episode_score)
        print("Episodic {} Score: {:.4f}".format(i_episode, episode_score))
        
        if max_score <= episode_score:                     
            max_score = episode_score
            agent.save(best_model_path)                     # save best model so far
        
        if len(total_rewards) >= 100:                       # record avg score for the latest 100 steps
            latest_avg_score = sum(total_rewards[(len(total_rewards)-100):]) / 100
            print("100 Episodic Everage Score: {:.4f}".format(latest_avg_score))
            avg_scores.append(latest_avg_score)
          
            if max_avg_score <= latest_avg_score:           # record better results
                worsen_tolerance = threshold_init           # re-count tolerance
                max_avg_score = latest_avg_score
            else:                                           
                if max_avg_score > 0.5:                     
                    worsen_tolerance -= 1                   # count worsening counts
                    print("Loaded from last best model.")
                    agent.load(best_model_path)             # continue from last best-model
                if worsen_tolerance <= 0:                   # earliy stop training
                    print("Early Stop Training.")
                    break
                    
    draw(total_rewards,"./training_score_plot.png", "Training Scores (Per Episode)")
    draw(avg_scores,"./training_100avgscore_plot.png", "Training Scores (Average of Latest 100 Episodes)", ylabel="Avg. Score")
    env.close()


/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Number of agents: 3
Episodic 1 Score: -5974.4032
Episodic 2 Score: -5110.2217
Episodic 3 Score: -1132.5765
Episodic 4 Score: -4569.9641
Episodic 5 Score: -8815.0356
Episodic 6 Score: -10095.5605
Episodic 7 Score: -8250.3529
Episodic 8 Score: -4136.9498
Episodic 9 Score: -4468.6437
Episodic 10 Score: -6881.8587
Episodic 11 Score: -7852.4505
Episodic 12 Score: -7616.1794
Episodic 13 Score: -9847.2477
Episodic 14 Score: -8719.5274
Episodic 15 Score: -10215.6273
Episodic 16 Score: -6633.6539
Episodic 17 Score: -7763.0295
Episodic 18 Score: -2730.4291
Episodic 19 Score: -5752.8390
Episodic 20 Score: -1763.2225
Episodic 21 Score: -5476.8554
Episodic 22 Score: -7680.0130
Episodic 23 Score: -5237.2468
Episodic 24 Score: -3696.1125
Episodic 25 Score: -2974.1834
Episodic 26 Score: -2774.8137
Episodic 27 Score: -2837.3390
Episodic 28 Score: -2744.8193
Episodic 29 Score: -2090.5613
Episodic 30 Score: -2094.5118
Episodic 31 Score: -2126.8882
Episodic 32 Score: -2330.5170
Episodic 33 Score: -2148.55

KeyboardInterrupt: ignored